In [6]:
import mysql.connector
from mysql.connector import Error

def check_mysql_connection(ip_address, username, password):
    try:
        # Establish the connection
        connection = mysql.connector.connect(
            host=ip_address,
            user=username,
            password=password
        )
        
        # If connection is successful
        if connection.is_connected():
            # Get server information
            server_info = connection.get_server_info()
            
            # Fetch current database and user info
            cursor = connection.cursor()
            cursor.execute("SELECT DATABASE();")
            result = cursor.fetchone()
            current_database = result[0] if result else "None"
            user = username
            
            return f"Connected to MySQL Server version {server_info}\n" \
                   f"Current Database: {current_database}\n" \
                   f"Connected User: {user}"
    except Error as e:
        return f"Error: Unable to connect to MySQL server.\n{e}"
    finally:
        if 'connection' in locals() and connection.is_connected():
            connection.close()

# Example usage
if __name__ == "__main__":
    ip = input("Enter MySQL Server IP: ")
    user = input("Enter MySQL Username: ")
    pwd = input("Enter MySQL Password: ")
    
    result = check_mysql_connection(ip, user, pwd)
    print(result)


Connected to MySQL Server version 9.1.0
Current Database: None
Connected User: dbigman


In [1]:
import logging
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError, OperationalError, ProgrammingError

# --------------------------------------------------------------------------
# Configure Logging
# --------------------------------------------------------------------------
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def check_mysql_connection(ip_address, username, password, database=None):
    """
    Checks connectivity to a MySQL server using SQLAlchemy 2.0+.
    
    :param ip_address: IP address or hostname of the MySQL server.
    :param username:   MySQL username.
    :param password:   MySQL password.
    :param database:   (Optional) Specific database to connect to.
    :return:           A message indicating the connection status and server info.
    """
    
    # Construct a database URL using the PyMySQL driver.
    db_url = f"mysql+pymysql://{username}:{password}@{ip_address}"
    if database:
        db_url += f"/{database}"
    
    try:
        # Create the Engine with basic connection-pooling parameters.
        # Adjust pool_size, max_overflow, and pool_recycle to suit your environment.
        engine = create_engine(
            db_url,
            echo=False,           # Turn on/off SQL statement logging
            pool_size=5,          # Size of the connection pool
            max_overflow=10,      # Connections allowed above pool_size
            pool_recycle=3600     # Recycle connections after this many seconds
        )
        
        # Attempt to connect.
        with engine.connect() as connection:
            # Get the server version (SQLAlchemy 2.0 style).
            server_version = connection.execute(text("SELECT VERSION()")).scalar_one_or_none()
            
            # Fetch the current database.
            current_db = connection.execute(text("SELECT DATABASE()")).scalar_one_or_none() or "None"
            
            # (Optional) Fetch the currently connected user.
            current_user = connection.execute(text("SELECT USER()")).scalar_one_or_none() or username
            
            return (
                f"Connected to MySQL Server version: {server_version}\n"
                f"Current Database: {current_db}\n"
                f"Connected User: {current_user}"
            )
    
    # ----------------------------------------------------------------------
    # Finer-grained error handling
    # ----------------------------------------------------------------------
    except OperationalError as oe:
        # OperationalError often indicates authentication issues or server unavailability.
        logging.error("OperationalError encountered: %s", oe)
        return "Error: Unable to connect to MySQL server (operational issue)."
    
    except ProgrammingError as pe:
        # ProgrammingError indicates issues like unknown database, syntax errors, etc.
        logging.error("ProgrammingError encountered: %s", pe)
        return "Error: Database issue encountered (programming error)."
    
    except SQLAlchemyError as e:
        # Catch-all for SQLAlchemy errors.
        logging.error("SQLAlchemyError encountered: %s", e)
        return "Error: An unexpected SQLAlchemy error occurred."
    
    except Exception as ex:
        # Catch-all for any non-SQLAlchemy exceptions.
        logging.error("Unexpected exception encountered: %s", ex)
        return "Error: An unexpected exception occurred."

# Example usage
if __name__ == "__main__":
    ip = input("Enter MySQL Server IP: ")
    user = input("Enter MySQL Username: ")
    pwd = input("Enter MySQL Password: ")
    
    result = check_mysql_connection(ip, user, pwd)
    print(result)


Connected to MySQL Server version: 9.1.0
Current Database: None
Connected User: dbigman@192.168.1.106
